# Project: Finding the Best Customers

In this project, I'll be using RFM, a customer segmentation technique that uses past purchase behavior that divide customers into groups in order to identify the top 10 customers.

## I. Online Retail Data

In [1]:
import pandas as pd

data = pd.read_excel("Online Retail.xlsx")
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


The data contains all customers transactions that occured between December 1, 2010 - December 9, 2011 for a online retailed based in the UK

Here's what each columns represent:
1. <font color=red>InvoiceNo</font>: Unique ID for every time a customer visits the store.
2. <font color=red>StockCode</font>: Unique ID for a product
3. <font color=red>Description</font>: Description of the product
4. <font color=red>Quantity</font>: Quantity of units bought
5. <font color=red>InvoiceDate</font>: Date and time of purchase
6. <font color=red>Unit Price</font>: Price per unit
7. <font color=red>CustomerID</font>: Unique ID for each customer
8. <font color=red>Country</font>: Country where transaction happened

## II. Exploring the Data

In this step, I'll be:
1. Explore if there are missing values
2. Explore the customers' distribution in each country
3. Invoice date should not be less or greater than the dates we specified
4. The unit price and quantity should be greater than 0

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

There are about 135,000 missing values in the CustomerID column, and since our analysis is based on customers, it's pointless to have transactions that we can't identify the customer

In [4]:
data = data[data["CustomerID"].notnull()]
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Now, there are no missing values in our data

In [5]:
data.shape

(406829, 8)

In [6]:
data = data[data["Quantity"]>0]
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


Now, we need to convert <font color=red>InvoiceDate</font> into datetime

In [8]:
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [9]:
data["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [10]:
data["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

So, our data date ranges are correct. 

Now, let's explore the customer distribution for each country

In [12]:
customer_country = data[["Country","CustomerID"]].drop_duplicates()
customer_country

,Country,CustomerID
0,United Kingdom,17850.0
9,United Kingdom,13047.0
26,France,12583.0
46,United Kingdom,13748.0
65,United Kingdom,15100.0
82,United Kingdom,15291.0
86,United Kingdom,14688.0
105,United Kingdom,17809.0
106,United Kingdom,15311.0
142,United Kingdom,16098.0


From this table, we can tell that there are about 4347 unique customers.

Let's group the table by country so we can determine the total number of unique customers each country has

In [16]:
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
35,United Kingdom,3921
14,Germany,94
13,France,87
30,Spain,30
3,Belgium,25
32,Switzerland,21
26,Portugal,19
18,Italy,14
12,Finland,12
1,Austria,11


Most of our customers were based in the UK. Since customer clusters are different by geography, it's better to just focus on one market at this time. So let's choose the UK

In [17]:
data = data[data["Country"] == "United Kingdom"]
data.shape

(354345, 8)

## III. RFM Customer Segmentation

Before we create our table, we need to identify which date to base our recency. Since our last invoice date was 2011-12-09, we can use 2011-12-10 to calculate it

In [18]:
import datetime 

now = datetime.datetime(2011,12,10)
now

datetime.datetime(2011, 12, 10, 0, 0)

Let's also add a new column for total price

In [19]:
data["TotalPrice"] = data["Quantity"] * data["UnitPrice"]
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


Let's create our RFM Table

In [23]:
rfm = data.groupby('CustomerID').agg({'InvoiceDate': lambda x: (now - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfm

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34
12821.0,214,6,92.72
12822.0,70,46,948.88
12823.0,74,5,1759.50
12824.0,59,25,397.12


In [24]:
rfm.dtypes

InvoiceDate      int64
InvoiceNo        int64
TotalPrice     float64
dtype: object

In [25]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': "Frequency",
    'TotalPrice': "MonetaryValue"
}, inplace=True)
rfm

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34
12821.0,214,6,92.72
12822.0,70,46,948.88
12823.0,74,5,1759.50
12824.0,59,25,397.12


### Split the metrics

We need to split the table into segments. We can do this with quartiles

In [26]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,MonetaryValue
0.25,17.0,17.0,300.04
0.50,50.0,41.0,651.82
0.75,142.0,99.0,1575.89


In [27]:
quantiles = quantiles.to_dict()
quantiles

{'Frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'MonetaryValue': {0.25: 300.03999999999996,
  0.5: 651.82000000000016,
  0.75: 1575.8900000000003},
 'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0}}

### Creating a segmented RFM Table

The best customers are the ones with the:
1. Lowest recency
2. Highest frequency
3. High monetary amount

In [29]:
def calc_r(x,p,d):
    if x<= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4
    
def calc_f_m(x,p,d):
    if x<= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
    
segment_rfm = rfm

segment_rfm["r_score"] = segment_rfm["Recency"].apply(calc_r, args=("Recency",quantiles))
segment_rfm["f_score"] = segment_rfm["Frequency"].apply(calc_f_m, args=("Frequency",quantiles))
segment_rfm["m_score"] = segment_rfm["MonetaryValue"].apply(calc_f_m, args=("MonetaryValue",quantiles))
segment_rfm.head(10)

,Recency,Frequency,MonetaryValue,r_score,f_score,m_score
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2
12821.0,214,6,92.72,4,4,4
12822.0,70,46,948.88,3,2,2
12823.0,74,5,1759.50,3,4,1
12824.0,59,25,397.12,3,3,3


Now, let's add a new column with the RFM score combined:

In [35]:
segment_rfm["RFMScore"] = segment_rfm["r_score"].astype(str)+ segment_rfm["f_score"].astype(str) + segment_rfm["m_score"].astype(str)
segment_rfm.head(10)

,Recency,Frequency,MonetaryValue,r_score,f_score,m_score,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122
12821.0,214,6,92.72,4,4,4,444
12822.0,70,46,948.88,3,2,2,322
12823.0,74,5,1759.50,3,4,1,341
12824.0,59,25,397.12,3,3,3,333


In [36]:
segment_rfm.dtypes

Recency            int64
Frequency          int64
MonetaryValue    float64
r_score            int64
f_score            int64
m_score            int64
RFMScore          object
dtype: object

### Top 20 Customers

In [37]:
segment_rfm[segment_rfm["RFMScore"] =='111'].sort_values('MonetaryValue', ascending=False).head(20)

,Recency,Frequency,MonetaryValue,r_score,f_score,m_score,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111
